<a href="https://colab.research.google.com/github/KartohaWhy/my_colab/blob/main/Copy_Nto_2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задача по предсказанию за кого из двух вариантов проголосует человек, исходя из похожести людей

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
!pip install catboost -q

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
sim = pd.read_csv('/content/drive/MyDrive/nto_2/3/sim.csv')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/nto_2/3/train.csv')

In [ ]:
sim.head()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df['class'].isna().sum()

In [ ]:
sim

In [ ]:
sim = sim.sort_values(by=['id1', 'id2']).reset_index(drop=True)

In [ ]:
sim.head()

In [ ]:
sim[sim['sim'] >= 0.9]

In [ ]:
sim['plus_minus'].value_counts()

In [ ]:
sim['plus_minus'] = sim['sim'].apply(lambda x: 1 if x > 0 else 0 if x < 0 else None)

In [ ]:
sim['more_90'] = sim['sim'].apply(lambda x: 1 if x >= 0.9 else 0 if x < 0.9 else None)

In [ ]:
sim['more_80'] = sim['sim'].apply(lambda x: 1 if x >= 0.8 else 0 )

In [ ]:
sim['more_90'] = sim['sim'].apply(lambda x: 1 if x >= 0.9 else 0)
sim['more_70'] = sim['sim'].apply(lambda x: 1 if (x >= 0.7) and (x < 0.9) else 0 )
sim['more_50'] = sim['sim'].apply(lambda x: 1 if (x >= 0.5) and (x < 0.7) else 0 )
sim['more_25'] = sim['sim'].apply(lambda x: 1 if (x >= 0.25) and (x < 0.5) else 0 )
sim['more_0'] = sim['sim'].apply(lambda x: 1 if (x >= 0) and (x < 0.25) else 0 )
sim['less_90'] = sim['sim'].apply(lambda x: 1 if x <= -0.9 else 0 )
sim['less_70'] = sim['sim'].apply(lambda x: 1 if (x <= -0.7) and (x>-0.9) else 0 )
sim['less_50'] = sim['sim'].apply(lambda x: 1 if (x <= -0.5) and (x>-0.7) else 0 )
sim['less_25'] = sim['sim'].apply(lambda x: 1 if (x <= -0.25) and (x>-0.5) else 0 )
sim['less_0'] = sim['sim'].apply(lambda x: 1 if (x < 0) and (x>-0.25) else 0 )

In [ ]:
df.head()

In [ ]:
sim.head()

In [ ]:
import numpy as np

In [ ]:
sim = sim.merge(df.rename(columns={'id': 'id1', 'class': 'vote_id1'}), on='id1', how='left')
sim = sim.merge(df.rename(columns={'id': 'id2', 'class': 'vote_id2'}), on='id2', how='left')

In [ ]:
sim.head()

In [ ]:
sim[sim['vote_id1']=='K']

In [ ]:
sim[sim['id1']==10000]

In [ ]:
import numpy as np

In [ ]:
conditions = [
    (sim['sim'] >= 0.9),
    (sim['sim'] >= 0.7) & (sim['sim'] < 0.9),
    (sim['sim'] >= 0.5) & (sim['sim'] < 0.7),
    (sim['sim'] >= 0.25) & (sim['sim'] < 0.5),
    (sim['sim'] >= 0) & (sim['sim'] < 0.25),
    (sim['sim'] <= -0.9),
    (sim['sim'] <= -0.7) & (sim['sim'] > -0.9),
    (sim['sim'] <= -0.5) & (sim['sim'] > -0.7),
    (sim['sim'] <= -0.25) & (sim['sim'] > -0.5),
    (sim['sim'] < 0) & (sim['sim'] > -0.25)
]

labels = ['more_90', 'more_70', 'more_50', 'more_25', 'more_0',
          'less_90', 'less_70', 'less_50', 'less_25', 'less_0']

for i, column in enumerate(labels):
    sim[column] = np.where(sim['vote_id2'].isna(), 0,
                     np.where(sim['vote_id2'] == 'K', 1,
                     np.where(sim['vote_id2'] == 'D', 2, 0)))
    sim[column] = np.where(conditions[i], sim[column], 0)

In [ ]:
grouped = sim.groupby('id1').agg({
    'vote_id1': 'first'  # оставляем vote_id1 (можно использовать 'first', 'max' и пр. на ваше усмотрение)
})

In [ ]:
grouped.head()

In [ ]:
labels_more = ['more_90', 'more_70', 'more_50', 'more_25', 'more_0']
labels_less = ['less_90', 'less_70', 'less_50', 'less_25', 'less_0']

In [ ]:
for column in labels_more:
    grouped[f'{column}_K'] = sim[sim[column] == 1].groupby('id1')[column].count() / 100
    grouped[f'{column}_D'] = sim[sim[column] == 2].groupby('id1')[column].count() / 100
    # grouped[f'{column}_NaN'] = sim[sim[column] == 0].groupby('id1')[column].count() / 100

In [ ]:
for column in labels_less:
    grouped[f'{column}_K'] = -sim[sim[column] == 1].groupby('id1')[column].count() / 100
    grouped[f'{column}_D'] = -sim[sim[column] == 2].groupby('id1')[column].count() / 100
    # grouped[f'{column}_NaN'] = sim[sim[column] == 0].groupby('id1')[column].count() / 100

In [ ]:
grouped.fillna(0, inplace=True)

In [ ]:
grouped.head(20)

In [ ]:
grouped.to_csv('grouped_nto_3.csv')

In [ ]:
train_grouped = grouped.iloc[0:10000].reset_index(drop=True)  # Строки с 0 по 9999
test_grouped = grouped.iloc[10000:11000].reset_index(drop=True)

In [ ]:
test_grouped.head()

In [ ]:
train_grouped.head()

In [ ]:
predictions_df = pd.DataFrame()
predictions_df['id'] = test_grouped['id1']

In [ ]:
predictions_df.head()

In [ ]:
test_grouped

In [ ]:
#test_grouped.drop(['vote_id1','id1'], axis=1, inplace=True)
test_grouped.drop(['vote_id1'], axis=1, inplace=True)

In [ ]:
#grouped['sum'] = grouped.loc[:, grouped.columns.difference(['id1', 'vote_id1'])].sum(axis=1)

In [ ]:
#grouped.reset_index(inplace=True)

In [ ]:
#sim.tail(50)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = train_grouped.drop(columns=['vote_id1','id1'])  # Убираем vote_id1 из X
y = train_grouped['vote_id1']  # Берем vote_id1 в y

# Используем train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
model = CatBoostClassifier(random_state=42,
                           thread_count=-1)

In [ ]:
model.fit(X_train, y_train,
          eval_set=(X_val, y_val),
          verbose=100, plot=False,
          early_stopping_rounds=100)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
y_pred = model.predict(X_val)

In [ ]:
accuracy = accuracy_score(y_val, y_pred)
accuracy

**Тут обучаем train + test**

In [ ]:
train_grouped.head()

In [ ]:
test_grouped.head()

In [ ]:
#X_train = train_grouped.drop(columns=['vote_id1', 'id1'])  # Убираем 'vote_id1' и 'id1' из train
X_train = train_grouped.drop(columns=['vote_id1'])  # Убираем 'vote_id1' и 'id1' из train
y_train = train_grouped['vote_id1']  # Берем 'vote_id1' в y

# Для тестовой выборки
X_test = test_grouped

In [ ]:
model_check = CatBoostClassifier(random_state=42,
                           thread_count=-1)

In [ ]:
model_check.fit(X_train, y_train,
          verbose=100, plot=False,
          early_stopping_rounds=100)

In [ ]:
pred = model_check.predict(X_test)

In [ ]:
pred.shape

In [ ]:
grouped_11000 = grouped.copy()

In [ ]:
grouped_11000.head()

In [ ]:
pred.shape

In [ ]:
grouped_11000.reset_index(inplace=True)

In [ ]:
grouped_11000.loc[(grouped_11000['id1'] >= 10000) & (grouped_11000['id1'] < 11000), 'vote_id1'] = pred

In [ ]:
#pred

In [ ]:
grouped_11000[grouped_11000['id1'] == 10001]

**Начинаем учить**

In [ ]:
sim = sim.copy()
sim = sim.sort_values(by=['id1', 'id2']).reset_index(drop=True)
sim = sim.merge(df.rename(columns={'id': 'id1', 'class': 'vote_id1'}), on='id1', how='left')
sim = sim.merge(df.rename(columns={'id': 'id2', 'class': 'vote_id2'}), on='id2', how='left')

In [ ]:
conditions = [
    (sim['sim'] >= 0.9),
    (sim['sim'] >= 0.7) & (sim['sim'] < 0.9),
    (sim['sim'] >= 0.5) & (sim['sim'] < 0.7),
    (sim['sim'] >= 0.25) & (sim['sim'] < 0.5),
    (sim['sim'] >= 0) & (sim['sim'] < 0.25),
    (sim['sim'] <= -0.9),
    (sim['sim'] <= -0.7) & (sim['sim'] > -0.9),
    (sim['sim'] <= -0.5) & (sim['sim'] > -0.7),
    (sim['sim'] <= -0.25) & (sim['sim'] > -0.5),
    (sim['sim'] < 0) & (sim['sim'] > -0.25)
]

labels = ['more_90', 'more_70', 'more_50', 'more_25', 'more_0',
          'less_90', 'less_70', 'less_50', 'less_25', 'less_0']

for i, column in enumerate(labels):
    sim[column] = np.where(sim['vote_id2'].isna(), 0,
                     np.where(sim['vote_id2'] == 'K', 1,
                     np.where(sim['vote_id2'] == 'D', 2, 0)))
    sim[column] = np.where(conditions[i], sim[column], 0)


grouped = sim.groupby('id1').agg({
    'vote_id1': 'first'  # оставляем vote_id1 (можно использовать 'first', 'max' и пр. на ваше усмотрение)
})

labels_more = ['more_90', 'more_70', 'more_50', 'more_25', 'more_0']
labels_less = ['less_90', 'less_70', 'less_50', 'less_25', 'less_0']

for column in labels_more:
    grouped[f'{column}_K'] = sim[sim[column] == 1].groupby('id1')[column].count() / 100
    grouped[f'{column}_D'] = sim[sim[column] == 2].groupby('id1')[column].count() / 100
    # grouped[f'{column}_NaN'] = sim[sim[column] == 0].groupby('id1')[column].count() / 100

for column in labels_less:
    grouped[f'{column}_K'] = -sim[sim[column] == 1].groupby('id1')[column].count() / 100
    grouped[f'{column}_D'] = -sim[sim[column] == 2].groupby('id1')[column].count() / 100
    # grouped[f'{column}_NaN'] = sim[sim[column] == 0].groupby('id1')[column].count() / 100

grouped.fillna(0, inplace=True)
grouped.reset_index(inplace=True)

train_grouped = grouped.iloc[0:10000].reset_index(drop=True)  # Строки с 0 по 9999
test_grouped = grouped.iloc[10000:11000].reset_index(drop=True)

# predictions_df = pd.DataFrame()
# predictions_df['id'] = test_grouped['id1']

In [ ]:
X_train = train_grouped.drop(columns=['vote_id1', 'id1'])  # Убираем 'vote_id1' и 'id1' из train
y_train = train_grouped['vote_id1']  # Берем 'vote_id1' в y

# Для тестовой выборки
X_test = test_grouped.drop(columns=['vote_id1', 'id1'])  # Убираем 'vote_id1' и 'id1' из test

model_check = CatBoostClassifier(random_state=42,
                           thread_count=-1)
model_check.fit(X_train, y_train,
          verbose=100, plot=False,
          early_stopping_rounds=100)

pred = model_check.predict(X_test)

In [ ]:
fi = model_check.get_feature_importance(prettified=True)
fi

**Заполнены 11000 строчек**

In [ ]:
df_11000 = df.copy()
new_ids = pd.DataFrame({'id': range(10000, 11000), 'class': pred})
df = pd.concat([df_11000, new_ids], ignore_index=True)
sim = sim.copy()
sim = sim.sort_values(by=['id1', 'id2']).reset_index(drop=True)
sim.drop(['vote_id1', 'vote_id2'], axis=1, inplace=True)
sim = sim.merge(df.rename(columns={'id': 'id1', 'class': 'vote_id1'}), on='id1', how='left')
sim = sim.merge(df.rename(columns={'id': 'id2', 'class': 'vote_id2'}), on='id2', how='left')

In [ ]:
#df

In [ ]:
#sim

In [ ]:
conditions = [
    (sim['sim'] >= 0.9),
    (sim['sim'] >= 0.7) & (sim['sim'] < 0.9),
    (sim['sim'] >= 0.5) & (sim['sim'] < 0.7),
    (sim['sim'] >= 0.25) & (sim['sim'] < 0.5),
    (sim['sim'] >= 0) & (sim['sim'] < 0.25),
    (sim['sim'] <= -0.9),
    (sim['sim'] <= -0.7) & (sim['sim'] > -0.9),
    (sim['sim'] <= -0.5) & (sim['sim'] > -0.7),
    (sim['sim'] <= -0.25) & (sim['sim'] > -0.5),
    (sim['sim'] < 0) & (sim['sim'] > -0.25)
]

labels = ['more_90', 'more_70', 'more_50', 'more_25', 'more_0',
          'less_90', 'less_70', 'less_50', 'less_25', 'less_0']

for i, column in enumerate(labels):
    sim[column] = np.where(sim['vote_id2'].isna(), 0,
                     np.where(sim['vote_id2'] == 'K', 1,
                     np.where(sim['vote_id2'] == 'D', 2, 0)))
    sim[column] = np.where(conditions[i], sim[column], 0)


grouped = sim.groupby('id1').agg({
    'vote_id1': 'first'  # оставляем vote_id1 (можно использовать 'first', 'max' и пр. на ваше усмотрение)
})

labels_more = ['more_90', 'more_70', 'more_50', 'more_25', 'more_0']
labels_less = ['less_90', 'less_70', 'less_50', 'less_25', 'less_0']

for column in labels_more:
    grouped[f'{column}_K'] = sim[sim[column] == 1].groupby('id1')[column].count() / 100
    grouped[f'{column}_D'] = sim[sim[column] == 2].groupby('id1')[column].count() / 100
    # grouped[f'{column}_NaN'] = sim[sim[column] == 0].groupby('id1')[column].count() / 100

for column in labels_less:
    grouped[f'{column}_K'] = -sim[sim[column] == 1].groupby('id1')[column].count() / 100
    grouped[f'{column}_D'] = -sim[sim[column] == 2].groupby('id1')[column].count() / 100
    # grouped[f'{column}_NaN'] = sim[sim[column] == 0].groupby('id1')[column].count() / 100

grouped.fillna(0, inplace=True)
grouped.reset_index(inplace=True)

train_grouped = grouped.iloc[0:11000].reset_index(drop=True)  # Строки с 0 по 9999
test_grouped = grouped.iloc[11000:12000].reset_index(drop=True)

# predictions_df = pd.DataFrame()
# predictions_df['id'] = test_grouped['id1']

In [ ]:
test_grouped.head()

In [ ]:
X_train = train_grouped.drop(columns=['vote_id1', 'id1'])  # Убираем 'vote_id1' и 'id1' из train
y_train = train_grouped['vote_id1']  # Берем 'vote_id1' в y

# Для тестовой выборки
X_test = test_grouped.drop(columns=['vote_id1', 'id1'])  # Убираем 'vote_id1' и 'id1' из test

model_check = CatBoostClassifier(random_state=42,
                           thread_count=-1)
model_check.fit(X_train, y_train,
          verbose=100, plot=False,
          early_stopping_rounds=100)

pred = model_check.predict(X_test)

In [ ]:
fi = model_check.get_feature_importance(prettified=True)
fi

In [ ]:
sim.shape

In [ ]:
df_12000 = df.copy()
new_ids = pd.DataFrame({'id': range(11000, 12000), 'class': pred})
df = pd.concat([df_12000, new_ids], ignore_index=True)
sim = sim.copy()
sim = sim.sort_values(by=['id1', 'id2']).reset_index(drop=True)
sim.drop(['vote_id1', 'vote_id2'], axis=1, inplace=True)
sim = sim.merge(df.rename(columns={'id': 'id1', 'class': 'vote_id1'}), on='id1', how='left')
sim = sim.merge(df.rename(columns={'id': 'id2', 'class': 'vote_id2'}), on='id2', how='left')

In [ ]:
#df

In [ ]:
sim.head()

In [ ]:
conditions = [
    (sim['sim'] >= 0.9),
    (sim['sim'] >= 0.7) & (sim['sim'] < 0.9),
    (sim['sim'] >= 0.5) & (sim['sim'] < 0.7),
    (sim['sim'] >= 0.25) & (sim['sim'] < 0.5),
    (sim['sim'] >= 0) & (sim['sim'] < 0.25),
    (sim['sim'] <= -0.9),
    (sim['sim'] <= -0.7) & (sim['sim'] > -0.9),
    (sim['sim'] <= -0.5) & (sim['sim'] > -0.7),
    (sim['sim'] <= -0.25) & (sim['sim'] > -0.5),
    (sim['sim'] < 0) & (sim['sim'] > -0.25)
]

labels = ['more_90', 'more_70', 'more_50', 'more_25', 'more_0',
          'less_90', 'less_70', 'less_50', 'less_25', 'less_0']

for i, column in enumerate(labels):
    sim[column] = np.where(sim['vote_id2'].isna(), 0,
                     np.where(sim['vote_id2'] == 'K', 1,
                     np.where(sim['vote_id2'] == 'D', 2, 0)))
    sim[column] = np.where(conditions[i], sim[column], 0)


grouped = sim.groupby('id1').agg({
    'vote_id1': 'first'  # оставляем vote_id1 (можно использовать 'first', 'max' и пр. на ваше усмотрение)
})

labels_more = ['more_90', 'more_70', 'more_50', 'more_25', 'more_0']
labels_less = ['less_90', 'less_70', 'less_50', 'less_25', 'less_0']

for column in labels_more:
    grouped[f'{column}_K'] = sim[sim[column] == 1].groupby('id1')[column].count() / 100
    grouped[f'{column}_D'] = sim[sim[column] == 2].groupby('id1')[column].count() / 100
    # grouped[f'{column}_NaN'] = sim[sim[column] == 0].groupby('id1')[column].count() / 100

for column in labels_less:
    grouped[f'{column}_K'] = -sim[sim[column] == 1].groupby('id1')[column].count() / 100
    grouped[f'{column}_D'] = -sim[sim[column] == 2].groupby('id1')[column].count() / 100
    # grouped[f'{column}_NaN'] = sim[sim[column] == 0].groupby('id1')[column].count() / 100

grouped.fillna(0, inplace=True)
grouped.reset_index(inplace=True)

train_grouped = grouped.iloc[0:12000].reset_index(drop=True)  # Строки с 0 по 9999
test_grouped = grouped.iloc[12000:13000].reset_index(drop=True)

predictions_df = pd.DataFrame()
predictions_df['id'] = test_grouped['id1']

In [ ]:
X_train = train_grouped.drop(columns=['vote_id1', 'id1'])  # Убираем 'vote_id1' и 'id1' из train
y_train = train_grouped['vote_id1']  # Берем 'vote_id1' в y

# Для тестовой выборки
X_test = test_grouped.drop(columns=['vote_id1', 'id1'])  # Убираем 'vote_id1' и 'id1' из test

model_check = CatBoostClassifier(random_state=42,
                           thread_count=-1)
model_check.fit(X_train, y_train,
          verbose=100, plot=False,
          early_stopping_rounds=100)

pred = model_check.predict(X_test)

In [ ]:
fi = model_check.get_feature_importance(prettified=True)
fi

**13000 заполненных**

In [ ]:
df_13000 = df.copy()
new_ids = pd.DataFrame({'id': range(12000, 13000), 'class': pred})
df = pd.concat([df_13000, new_ids], ignore_index=True)
sim = sim.copy()
sim = sim.sort_values(by=['id1', 'id2']).reset_index(drop=True)
sim.drop(['vote_id1', 'vote_id2'], axis=1, inplace=True)
sim = sim.merge(df.rename(columns={'id': 'id1', 'class': 'vote_id1'}), on='id1', how='left')
sim = sim.merge(df.rename(columns={'id': 'id2', 'class': 'vote_id2'}), on='id2', how='left')

In [ ]:
df.tail()

In [ ]:
conditions = [
    (sim['sim'] >= 0.9),
    (sim['sim'] >= 0.7) & (sim['sim'] < 0.9),
    (sim['sim'] >= 0.5) & (sim['sim'] < 0.7),
    (sim['sim'] >= 0.25) & (sim['sim'] < 0.5),
    (sim['sim'] >= 0) & (sim['sim'] < 0.25),
    (sim['sim'] <= -0.9),
    (sim['sim'] <= -0.7) & (sim['sim'] > -0.9),
    (sim['sim'] <= -0.5) & (sim['sim'] > -0.7),
    (sim['sim'] <= -0.25) & (sim['sim'] > -0.5),
    (sim['sim'] < 0) & (sim['sim'] > -0.25)
]

labels = ['more_90', 'more_70', 'more_50', 'more_25', 'more_0',
          'less_90', 'less_70', 'less_50', 'less_25', 'less_0']

for i, column in enumerate(labels):
    sim[column] = np.where(sim['vote_id2'].isna(), 0,
                     np.where(sim['vote_id2'] == 'K', 1,
                     np.where(sim['vote_id2'] == 'D', 2, 0)))
    sim[column] = np.where(conditions[i], sim[column], 0)


grouped = sim.groupby('id1').agg({
    'vote_id1': 'first'  # оставляем vote_id1 (можно использовать 'first', 'max' и пр. на ваше усмотрение)
})

labels_more = ['more_90', 'more_70', 'more_50', 'more_25', 'more_0']
labels_less = ['less_90', 'less_70', 'less_50', 'less_25', 'less_0']

for column in labels_more:
    grouped[f'{column}_K'] = sim[sim[column] == 1].groupby('id1')[column].count() / 100
    grouped[f'{column}_D'] = sim[sim[column] == 2].groupby('id1')[column].count() / 100
    # grouped[f'{column}_NaN'] = sim[sim[column] == 0].groupby('id1')[column].count() / 100

for column in labels_less:
    grouped[f'{column}_K'] = -sim[sim[column] == 1].groupby('id1')[column].count() / 100
    grouped[f'{column}_D'] = -sim[sim[column] == 2].groupby('id1')[column].count() / 100
    # grouped[f'{column}_NaN'] = sim[sim[column] == 0].groupby('id1')[column].count() / 100

grouped.fillna(0, inplace=True)
grouped.reset_index(inplace=True)

train_grouped = grouped.iloc[0:13000].reset_index(drop=True)  # Строки с 0 по 9999
test_grouped = grouped.iloc[13000:14000].reset_index(drop=True)

predictions_df = pd.DataFrame()
predictions_df['id'] = test_grouped['id1']

In [ ]:
X_train = train_grouped.drop(columns=['vote_id1', 'id1'])  # Убираем 'vote_id1' и 'id1' из train
y_train = train_grouped['vote_id1']  # Берем 'vote_id1' в y

# Для тестовой выборки
X_test = test_grouped.drop(columns=['vote_id1', 'id1'])  # Убираем 'vote_id1' и 'id1' из test

model_check = CatBoostClassifier(random_state=42,
                           thread_count=-1)
model_check.fit(X_train, y_train,
          verbose=100, plot=False,
          early_stopping_rounds=100)

pred = model_check.predict(X_test)

In [ ]:
fi = model_check.get_feature_importance(prettified=True)
fi

**14000 заполненных**

In [ ]:
df_14000 = df.copy()
new_ids = pd.DataFrame({'id': range(13000, 14000), 'class': pred})
df = pd.concat([df_14000, new_ids], ignore_index=True)
sim = sim.copy()
sim = sim.sort_values(by=['id1', 'id2']).reset_index(drop=True)
sim.drop(['vote_id1', 'vote_id2'], axis=1, inplace=True)
sim = sim.merge(df.rename(columns={'id': 'id1', 'class': 'vote_id1'}), on='id1', how='left')
sim = sim.merge(df.rename(columns={'id': 'id2', 'class': 'vote_id2'}), on='id2', how='left')

In [ ]:
df.tail()

In [ ]:
conditions = [
    (sim['sim'] >= 0.9),
    (sim['sim'] >= 0.7) & (sim['sim'] < 0.9),
    (sim['sim'] >= 0.5) & (sim['sim'] < 0.7),
    (sim['sim'] >= 0.25) & (sim['sim'] < 0.5),
    (sim['sim'] >= 0) & (sim['sim'] < 0.25),
    (sim['sim'] <= -0.9),
    (sim['sim'] <= -0.7) & (sim['sim'] > -0.9),
    (sim['sim'] <= -0.5) & (sim['sim'] > -0.7),
    (sim['sim'] <= -0.25) & (sim['sim'] > -0.5),
    (sim['sim'] < 0) & (sim['sim'] > -0.25)
]

labels = ['more_90', 'more_70', 'more_50', 'more_25', 'more_0',
          'less_90', 'less_70', 'less_50', 'less_25', 'less_0']

for i, column in enumerate(labels):
    sim[column] = np.where(sim['vote_id2'].isna(), 0,
                     np.where(sim['vote_id2'] == 'K', 1,
                     np.where(sim['vote_id2'] == 'D', 2, 0)))
    sim[column] = np.where(conditions[i], sim[column], 0)


grouped = sim.groupby('id1').agg({
    'vote_id1': 'first'  # оставляем vote_id1 (можно использовать 'first', 'max' и пр. на ваше усмотрение)
})

labels_more = ['more_90', 'more_70', 'more_50', 'more_25', 'more_0']
labels_less = ['less_90', 'less_70', 'less_50', 'less_25', 'less_0']

for column in labels_more:
    grouped[f'{column}_K'] = sim[sim[column] == 1].groupby('id1')[column].count() / 100
    grouped[f'{column}_D'] = sim[sim[column] == 2].groupby('id1')[column].count() / 100
    # grouped[f'{column}_NaN'] = sim[sim[column] == 0].groupby('id1')[column].count() / 100

for column in labels_less:
    grouped[f'{column}_K'] = -sim[sim[column] == 1].groupby('id1')[column].count() / 100
    grouped[f'{column}_D'] = -sim[sim[column] == 2].groupby('id1')[column].count() / 100
    # grouped[f'{column}_NaN'] = sim[sim[column] == 0].groupby('id1')[column].count() / 100

grouped.fillna(0, inplace=True)
grouped.reset_index(inplace=True)

train_grouped = grouped.iloc[0:14000].reset_index(drop=True)  # Строки с 0 по 9999
test_grouped = grouped.iloc[14000:15000].reset_index(drop=True)

predictions_df = pd.DataFrame()
predictions_df['id'] = test_grouped['id1']

In [ ]:
X_train = train_grouped.drop(columns=['vote_id1', 'id1'])  # Убираем 'vote_id1' и 'id1' из train
y_train = train_grouped['vote_id1']  # Берем 'vote_id1' в y

# Для тестовой выборки
X_test = test_grouped.drop(columns=['vote_id1', 'id1'])  # Убираем 'vote_id1' и 'id1' из test

model_check = CatBoostClassifier(random_state=42,
                           thread_count=-1)
model_check.fit(X_train, y_train,
          verbose=100, plot=False,
          early_stopping_rounds=100)

pred = model_check.predict(X_test)

In [ ]:
fi = model_check.get_feature_importance(prettified=True)
fi

In [ ]:
df_15000 = df.copy()
new_ids = pd.DataFrame({'id': range(14000, 15000), 'class': pred})
df = pd.concat([df_15000, new_ids], ignore_index=True)
sim = sim.copy()
sim = sim.sort_values(by=['id1', 'id2']).reset_index(drop=True)
sim.drop(['vote_id1', 'vote_id2'], axis=1, inplace=True)
sim = sim.merge(df.rename(columns={'id': 'id1', 'class': 'vote_id1'}), on='id1', how='left')
sim = sim.merge(df.rename(columns={'id': 'id2', 'class': 'vote_id2'}), on='id2', how='left')

In [ ]:
conditions = [
    (sim['sim'] >= 0.9),
    (sim['sim'] >= 0.7) & (sim['sim'] < 0.9),
    (sim['sim'] >= 0.5) & (sim['sim'] < 0.7),
    (sim['sim'] >= 0.25) & (sim['sim'] < 0.5),
    (sim['sim'] >= 0) & (sim['sim'] < 0.25),
    (sim['sim'] <= -0.9),
    (sim['sim'] <= -0.7) & (sim['sim'] > -0.9),
    (sim['sim'] <= -0.5) & (sim['sim'] > -0.7),
    (sim['sim'] <= -0.25) & (sim['sim'] > -0.5),
    (sim['sim'] < 0) & (sim['sim'] > -0.25)
]

labels = ['more_90', 'more_70', 'more_50', 'more_25', 'more_0',
          'less_90', 'less_70', 'less_50', 'less_25', 'less_0']

for i, column in enumerate(labels):
    sim[column] = np.where(sim['vote_id2'].isna(), 0,
                     np.where(sim['vote_id2'] == 'K', 1,
                     np.where(sim['vote_id2'] == 'D', 2, 0)))
    sim[column] = np.where(conditions[i], sim[column], 0)


grouped = sim.groupby('id1').agg({
    'vote_id1': 'first'  # оставляем vote_id1 (можно использовать 'first', 'max' и пр. на ваше усмотрение)
})

labels_more = ['more_90', 'more_70', 'more_50', 'more_25', 'more_0']
labels_less = ['less_90', 'less_70', 'less_50', 'less_25', 'less_0']

for column in labels_more:
    grouped[f'{column}_K'] = sim[sim[column] == 1].groupby('id1')[column].count() / 100
    grouped[f'{column}_D'] = sim[sim[column] == 2].groupby('id1')[column].count() / 100
    # grouped[f'{column}_NaN'] = sim[sim[column] == 0].groupby('id1')[column].count() / 100

for column in labels_less:
    grouped[f'{column}_K'] = -sim[sim[column] == 1].groupby('id1')[column].count() / 100
    grouped[f'{column}_D'] = -sim[sim[column] == 2].groupby('id1')[column].count() / 100
    # grouped[f'{column}_NaN'] = sim[sim[column] == 0].groupby('id1')[column].count() / 100

grouped.fillna(0, inplace=True)
grouped.reset_index(inplace=True)

In [ ]:
grouped[grouped['id1']==14991]

In [ ]:
df.tail()

In [ ]:
predictions = df.loc[10000:14999]

In [ ]:
predictions.to_csv('nto_3_cb_learn_1000.csv', index=False)

In [ ]:
df[df['id']==12000]

In [ ]:
df.shape

In [ ]:
pred.shape

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier

In [ ]:
train_grouped = grouped.iloc[0:11000].reset_index(drop=True)  # Строки с 0 по 9999
test_grouped = grouped.iloc[11000:12000].reset_index(drop=True)

In [ ]:
grouped.head()

In [ ]:
grouped.reset_index(inplace=True)

In [ ]:
grouped[grouped['id1']==10999]

In [ ]:
grouped.drop('sum', axis=1, inplace=True)

In [ ]:
predictions_df['class'] = pred

In [ ]:
predictions_df.head()

In [ ]:
predictions_df.to_csv('nto_3_cb_1.csv', index=False)

**Ниже не лезть**

In [ ]:
count_votes = sim.groupby('id1')['vote_id2'].value_counts().unstack(fill_value=0)
count_votes

In [ ]:
aggregate = sim.groupby(['id1', 'vote_id2'])[['plus_minus', 'more_90', 'more_70', 'more_50', 'more_25', 'more_0',
                                                'less_90', 'less_70', 'less_50', 'less_25', 'less_0']].sum().reset_index()

In [ ]:
aggregate

In [ ]:
aggregate = aggregate.pivot(index='id1', columns='vote_id2', values=list(aggregate.columns[2:]))
aggregate.head()

In [ ]:
count_votes = sim.groupby('id1')['vote_id2'].value_counts().unstack(fill_value=0)

In [ ]:
for column in ['plus_minus', 'more_90', 'more_70', 'more_50', 'more_25', 'more_0',
               'less_90', 'less_70', 'less_50', 'less_25', 'less_0']:
    # Проверяем, есть ли данные и считаем соотношение
    if 'D' in aggregate and 'K' in aggregate:
        aggregate[f"{column}_D_ratio"] = aggregate[column]['D'] / count_votes['D']
        aggregate[f"{column}_K_ratio"] = aggregate[column]['K'] / count_votes['K']

In [ ]:
aggregate.head()

In [ ]:
total_counts = sim['id1'].value_counts()

In [ ]:
for column in ['plus_minus', 'more_90', 'more_70', 'more_50', 'more_25', 'more_0',
               'less_90', 'less_70', 'less_50', 'less_25', 'less_0']:
    # Добавляем новые столбцы для K и D
    sim[f"{column}_K"] = sim['id1'].map(lambda x: count_votes.loc[x, 'K'] / total_counts[x] if x in total_counts else 0)
    sim[f"{column}_D"] = sim['id1'].map(lambda x: count_votes.loc[x, 'D'] / total_counts[x] if x in total_counts else 0)

# Выведем результат
print(sim.head())

In [ ]:
sim[['less_90_K','less_25_K', 'less_70_K',  'less_0_K', 'less_50_K','more_90_K','more_25_K', 'more_70_K',  'more_0_K', 'more_50_K']]

In [ ]:
# sim['less_90'] = sim['sim'].apply(lambda x: 1 if x <= -0.9 else 0 )
# sim['less_80'] = sim['sim'].apply(lambda x: 1 if x <= -0.9 else 0 )
# sim['less_70'] = sim['sim'].apply(lambda x: 1 if x <= -0.9 else 0 )
# sim['less_60'] = sim['sim'].apply(lambda x: 1 if x <= -0.9 else 0 )
# sim['less_50'] = sim['sim'].apply(lambda x: 1 if x <= -0.9 else 0 )
# sim['less_40'] = sim['sim'].apply(lambda x: 1 if x <= -0.9 else 0 )
# sim['less_30'] = sim['sim'].apply(lambda x: 1 if x <= -0.9 else 0 )
# sim['less_20'] = sim['sim'].apply(lambda x: 1 if x <= -0.9 else 0 )
# sim['less_10'] = sim['sim'].apply(lambda x: 1 if x <= -0.9 else 0 )

In [ ]:
sim.head()

In [ ]:
sim.head()

In [ ]:
df

In [ ]:
df['class'].value_counts()